In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras import Sequential, layers
import os
import numpy as np
import pandas as pd

In [4]:
IMG_SIZE = 180

resize_and_rescale = tf.keras.Sequential([
  layers.Resizing(IMG_SIZE, IMG_SIZE),
  layers.Rescaling(1./255)
])

In [6]:
def process_csv_file(file):
    csv_data = pd.read_csv(file)
    csv_data = csv_data.drop("class", axis = 1)
    return (csv_data.to_numpy())

In [7]:
def set_padding(arr, max_length):
    arr = np.append(arr, np.zeros((max_length-arr.shape[0],300)), axis=0)
    return np.expand_dims(arr, 0)

In [8]:
def prepare_data(folder_path, labels):
    total = 0
    max_length = 0
    # find max length
    for folder in os.listdir(folder_path):
        if folder != ".DS_Store":
            total += len(os.listdir(os.path.join(folder_path, folder)))
            for file in os.listdir(os.path.join(folder_path, folder)):
                data = process_csv_file(os.path.join(folder_path, folder, file))
                if data.shape[0] > max_length:
                    max_length = data.shape[0]
    # Make the arrays
    X = np.empty((0, max_length, 300))
    Y = np.empty((0,), int)
    for folder in os.listdir(folder_path):
        print(folder)
        if folder != ".DS_Store":
            for file in os.listdir(os.path.join(folder_path, folder)):
                data = process_csv_file(os.path.join(folder_path, folder, file))
                data = set_padding(data, max_length)
                X = np.vstack((X, data))
                Y = np.append(Y, labels[folder])
    print(total, max_length)
    return X, Y


In [9]:
folder_path = "/Users/aly/Documents/Programming/Apps/Machine Learning/ASL Converter/training_models/mediapipe/reformatting-the-data/data_four_labels/"
labels= {"coffee": 0, 'dog': 1, 'milk': 2, 'door': 3}
X, Y = prepare_data(folder_path, labels)

.DS_Store
dog
milk
coffee
door
143 211


In [10]:
# split data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

In [11]:
X_train.shape, y_train.shape

((114, 211, 300), (114,))

DATA AUGMENTATION

In [12]:
data_augmentation = tf.keras.Sequential([
  layers.RandomFlip("horizontal_and_vertical"),
  layers.RandomRotation(
    factor=0.25,
    fill_mode='nearest',
    interpolation='nearest',
),
layers.RandomTranslation(
    height_factor=(-0.2, 0.3),
    width_factor=(-0.2, 0.3),
    fill_mode='nearest',
    interpolation='nearest',
)
])

In [13]:
batch_size = 32
AUTOTUNE = tf.data.AUTOTUNE

def prepare(ds, shuffle=False, augment=False):

  if shuffle:
    ds = ds.shuffle(1000)

  # Batch all datasets.
  ds = ds.batch(batch_size)

  # Use data augmentation only on the training set.
  if augment:
    ds = ds.map(lambda x, y: (data_augmentation(x, training=True), y), 
                num_parallel_calls=AUTOTUNE)

  # Use buffered prefetching on all datasets.
  return ds.prefetch(buffer_size=AUTOTUNE)

In [14]:
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_ds = prepare(train_ds, shuffle=True, augment=True)
test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test))
test_ds = prepare(test_ds)

In [15]:
train_ds, train_ds

(<PrefetchDataset element_spec=(TensorSpec(shape=(None, 211, 300), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>,
 <PrefetchDataset element_spec=(TensorSpec(shape=(None, 211, 300), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>)

In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

model = Sequential()
model.add(LSTM(64, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(64, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(64))
model.add(Dense(4, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])




In [17]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

model.compile(loss=SparseCategoricalCrossentropy(), optimizer=Adam(), metrics=["accuracy"])
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 211, 64)           93440     
                                                                 
 dropout (Dropout)           (None, 211, 64)           0         
                                                                 
 lstm_1 (LSTM)               (None, 211, 64)           33024     
                                                                 
 dropout_1 (Dropout)         (None, 211, 64)           0         
                                                                 
 lstm_2 (LSTM)               (None, 64)                33024     
                                                                 
 dense (Dense)               (None, 4)                 260       
                                                                 
Total params: 159,748
Trainable params: 159,748
Non-tr

In [18]:
history = model.fit(train_ds, epochs=50, batch_size=32, validation_data=test_ds)

InvalidArgumentError: Cannot assign a device for operation sequential_3/random_flip/stateful_uniform_full_int/RngReadAndSkip: Could not satisfy explicit device specification '' because the node {{colocation_node sequential_3/random_flip/stateful_uniform_full_int/RngReadAndSkip}} was colocated with a group of nodes that required incompatible device '/job:localhost/replica:0/task:0/device:GPU:0'. All available devices [/job:localhost/replica:0/task:0/device:CPU:0, /job:localhost/replica:0/task:0/device:GPU:0]. 
Colocation Debug Info:
Colocation group had the following types and supported devices: 
Root Member(assigned_device_name_index_=2 requested_device_name_='/job:localhost/replica:0/task:0/device:GPU:0' assigned_device_name_='/job:localhost/replica:0/task:0/device:GPU:0' resource_device_name_='/job:localhost/replica:0/task:0/device:GPU:0' supported_device_types_=[CPU] possible_devices_=[]
RngReadAndSkip: CPU 
_Arg: GPU CPU 

Colocation members, user-requested devices, and framework a